In [1]:
import pandas as pd
import numpy as np

In [2]:
df = pd.read_csv('/content/matches.csv')
df

,id,season,city,date,team1,team2,toss_winner,toss_decision,result,dl_applied,winner,win_by_runs,win_by_wickets,player_of_match,venue,umpire1,umpire2,umpire3
0,1,2017,Hyderabad,2017-04-05,Sunrisers Hyderabad,Royal Challengers Bangalore,Royal Challengers Bangalore,field,normal,0,Sunrisers Hyderabad,35,0,Yuvraj Singh,"Rajiv Gandhi International Stadium, Uppal",AY Dandekar,NJ Llong,NaN
1,2,2017,Pune,2017-04-06,Mumbai Indians,Rising Pune Supergiant,Rising Pune Supergiant,field,normal,0,Rising Pune Supergiant,0,7,SPD Smith,Maharashtra Cricket Association Stadium,A Nand Kishore,S Ravi,NaN
2,3,2017,Rajkot,2017-04-07,Gujarat Lions,Kolkata Knight Riders,Kolkata Knight Riders,field,normal,0,Kolkata Knight Riders,0,10,CA Lynn,Saurashtra Cricket Association Stadium,Nitin Menon,CK Nandan,NaN
3,4,2017,Indore,2017-04-08,Rising Pune Supergiant,Kings XI Punjab,Kings XI Punjab,field,normal,0,Kings XI Punjab,0,6,GJ Maxwell,Holkar Cricket Stadium,AK Chaudhary,C Shamshuddin,NaN
4,5,2017,Bangalore,2017-04-08,Royal Challengers Bangalore,Delhi Daredevils,Royal Challengers Bangalore,bat,normal,0,Royal Challengers Bangalore,15,0,KM Jadhav,M Chinnaswamy Stadium,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
751,11347,2019,Mumbai,05/05/19,Kolkata Knight Riders,Mumbai Indians,Mumbai Indians,field,normal,0,Mumbai Indians,0,9,HH Pandya,Wankhede Stadium,Nanda Kishore,O Nandan,S Ravi
752,11412,2019,Chennai,07/05/19,Chennai Super Kings,Mumbai Indians,Chennai Super Kings,bat,normal,0,Mumbai Indians,0,6,AS Yadav,M. A. Chidambaram Stadium,Nigel Llong,Nitin Menon,Ian Gould
753,11413,2019,Visakhapatnam,08/05/19,Sunrisers Hyderabad,Delhi Capitals,Delhi Capitals,field,normal,0,Delhi Capitals,0,2,RR Pant,ACA-VDCA Stadium,NaN,NaN,NaN
754,11414,2019,Visakhapatnam,10/05/19,Delhi Capitals,Chennai Super Kings,Chennai Super Kings,field,normal,0,Chennai Super Kings,0,6,F du Plessis,ACA-VDCA Stadium,Sundaram Ravi,Bruce Oxenford,Chettithody Shamshuddin


In [3]:
final_df = df[['team1', 'team2', 'toss_decision', 'toss_winner', 'winner']]
final_df.head(5)

,team1,team2,toss_decision,toss_winner,winner
0,Sunrisers Hyderabad,Royal Challengers Bangalore,field,Royal Challengers Bangalore,Sunrisers Hyderabad
1,Mumbai Indians,Rising Pune Supergiant,field,Rising Pune Supergiant,Rising Pune Supergiant
2,Gujarat Lions,Kolkata Knight Riders,field,Kolkata Knight Riders,Kolkata Knight Riders
3,Rising Pune Supergiant,Kings XI Punjab,field,Kings XI Punjab,Kings XI Punjab
4,Royal Challengers Bangalore,Delhi Daredevils,bat,Royal Challengers Bangalore,Royal Challengers Bangalore


In [4]:
final_df.describe()

,team1,team2,toss_decision,toss_winner,winner
count,756,756,756,756,752
unique,15,15,2,15,15
top,Mumbai Indians,Kolkata Knight Riders,field,Mumbai Indians,Mumbai Indians
freq,101,95,463,98,109


In [5]:
final_df.dropna(inplace = True)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [6]:
teams = {}
count = 0
for i in range(len(df)):
  if df.loc[i]['team1'] not in teams:
    teams[df.loc[i]['team1']] = count
    count += 1

  if df.loc[i]['team2'] not in teams:
    teams[df.loc[i]['team2']] = count
    count += 1

In [7]:
x = final_df[['team1', 'team2', 'toss_decision', 'toss_winner']]
y = final_df[['winner']]

In [8]:
x

,team1,team2,toss_decision,toss_winner
0,Sunrisers Hyderabad,Royal Challengers Bangalore,field,Royal Challengers Bangalore
1,Mumbai Indians,Rising Pune Supergiant,field,Rising Pune Supergiant
2,Gujarat Lions,Kolkata Knight Riders,field,Kolkata Knight Riders
3,Rising Pune Supergiant,Kings XI Punjab,field,Kings XI Punjab
4,Royal Challengers Bangalore,Delhi Daredevils,bat,Royal Challengers Bangalore
...,...,...,...,...
751,Kolkata Knight Riders,Mumbai Indians,field,Mumbai Indians
752,Chennai Super Kings,Mumbai Indians,bat,Chennai Super Kings
753,Sunrisers Hyderabad,Delhi Capitals,field,Delhi Capitals
754,Delhi Capitals,Chennai Super Kings,field,Chennai Super Kings


In [9]:
x = np.array(x)
y = np.array(y)

In [10]:
for i in range(len(x)):
  x[i][0] = teams[x[i][0]]
  x[i][1] = teams[x[i][1]]
  x[i][3] = teams[x[i][3]]

  y[i][0] = teams[y[i][0]]

In [11]:
x

array([[0, 1, 'field', 1],
       [2, 3, 'field', 3],
       [4, 5, 'field', 5],
       ...,
       [0, 14, 'field', 14],
       [14, 8, 'field', 8],
       [2, 8, 'bat', 2]], dtype=object)

In [12]:
field_bat = {'field':0, 'bat':1}
for i in range(len(x)):
  x[i][2] = field_bat[x[i][2]]

In [13]:
x

array([[0, 1, 0, 1],
       [2, 3, 0, 3],
       [4, 5, 0, 5],
       ...,
       [0, 14, 0, 14],
       [14, 8, 0, 8],
       [2, 8, 1, 2]], dtype=object)

In [14]:
# toss
for i in range(len(x)):
  if x[i][3] == x[i][0]:
    x[i][3] == 0
  else:
    x[i][3] == 1

In [15]:
ones = 0
for i in range(len(y)):
  if y[i][0] == x[i][1]:
    if ones < 370:
      ones += 1
      y[i][0] = 1

    else:
      z = x[i][0]
      x[i][0] = x[i][1]
      x[i][1] = z

      y[i][0] = 0
  
  else:
    y[i][0] = 0      
    

In [16]:
x = np.array(x, dtype = 'int32')
y = np.array(y, dtype = 'int32')

In [17]:
y = y.ravel()

In [18]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.2, random_state = 42)

In [19]:
from sklearn.linear_model import LogisticRegression
lr = LogisticRegression()
model = lr.fit(x_train, y_train)
model.score(x_test, y_test)

0.5231788079470199

In [31]:
from sklearn.ensemble import RandomForestClassifier
rf = RandomForestClassifier()
model2 = rf.fit(x_train, y_train)
model2.score(x_test, y_test)

0.6158940397350994

In [21]:
from sklearn.svm import SVC
model3 = SVC().fit(x_train, y_train)
model3.score(x_test, y_test)

0.5496688741721855

In [38]:
test = np.array([2,1,0,1]).reshape(1,-1)
model.predict(test)
model2.predict(test)
model3.predict(test)

array([1], dtype=int32)

In [37]:
import pickle as pkl
with open ('model.pkl', 'wb') as f:
  pkl.dump(model2, f)